<div style="text-align: right"><strong>Capstone #3:</strong> <span style="color:darkred">Supervised Learning</span> </div>

<a id="top"></a>

#### <span style="color:darkred">__Part 1: Data Exploration__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-exploration.ipynb </span><br><br><span style="color:darkred">__Part 2: Models__ https://github.com/kimrharper/thinkful/blob/master/unit3/unit3-capstone-models.ipynb </span>

----

# <span style="color:darkred">Part 2: </span><span style="color:darkblue">L1 Prediction from ELL Writing Samples</span>

__Author:__ Ryan Harper 

----

<a href='#ov'>Overview</a><br>
<a href='#exp'>Experiment</a><br>
<a href='#sec1'>1. Models:</a><br>
><a href='#seca'>A. LR - Ordinary Least Squares</a><br>
<a href='#secb'>B. LR - Logistic Regression</a> <a href='#secb1'> (Lasso)</a> <a href='#secb2'> (Ridge)</a><br>
<a href='#secc'>C. NN - K Nearest Neighbors</a><br>
<a href='#secd'>D. NN - Naive Bayes</a><br>
<a href='#sece'>E. NN - Decision Tree</a><br>
<a href='#secf'>F. Ensemble - Random Forest</a><br>

<a href='#sec2'>2. Model Comparison</a><br>

<a id="ov"></a>

<a id="sec1"></a>

# <span style="color:darkblue">1. Models:</span>  <a href='#top'>(top)</a>

In [33]:
# iPython/Jupyter Notebook
import time
from pprint import pprint
import warnings
from IPython.display import Image

import time

# Data processing
import pandas as pd
import plotly as plo
import seaborn as sns
from scipy import stats
from collections import Counter
import numpy as np
import itertools

# NLP
from nltk.corpus import stopwords as sw
from nltk.util import ngrams
from nltk.corpus import brown
import nltk
import re
from nltk.tokenize import RegexpTokenizer
import difflib

# Stats
from sklearn.metrics import classification_report, roc_curve,roc_auc_score,accuracy_score
from sklearn import metrics

# Preparing Models
from sklearn.model_selection import train_test_split

# Decomposition
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix

# Models
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.naive_bayes import BernoulliNB,MultinomialNB,GaussianNB

# Ensemble
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

#Visualization
from IPython.display import Image
import pydotplus
import graphviz

# import altair as alt

__Import Features + Target__

In [34]:
features = pd.read_csv('blogfeatures.csv').sample(frac=1.0)
del features['Unnamed: 0']
del features['id']
del features['content']
del features['pos']
del features['pos2']
del features['pos3']
del features['tokens']
del features['nns']

In [35]:
features.language.value_counts()

Japanese               9536
Traditional Chinese    3407
Korean                 1122
English                  65
Name: language, dtype: int64

__Declare X,y Variables__

In [36]:
y = features['language'].values.reshape(-1, 1).ravel()
X = features[features.columns[~features.columns.str.contains('language')]]
X.head()

print(np.shape(y))
print(np.shape(X))

(14130,)
(14130, 15450)


###  <span style="color:darkblue">C. Statistical Significance <a href='#top'>(top)</a>

In [37]:
from scipy.stats import median_test,mannwhitneyu,f_oneway
def mw_test(a,b):
    stat,p = mannwhitneyu(a,b, use_continuity=True, alternative=None)
    return stat,p

def moods_median_test(a,b,c,d):
    stat, p, med, tbl = median_test(a,b,c,d)
    return stat,p

def f1way_test(a,b,c,d):
    f,b = f_oneway(a,b,c,d)
    return f,b

import warnings
warnings.filterwarnings('ignore')

__B. Mood’s Median test (2+ Non-Normally Distributed Independent Samples)__

In [38]:
exclude = ['id','content','language','tokens','pos','pos2','pos3','nns']

In [39]:
lang = list(features.language.unique())

In [40]:
moodslist = {}
for c in features.columns: 
    if c not in exclude: 
        g = [(features[c][features.language == l]) for l in lang]
        stat,p = moods_median_test(g[0],g[1],g[2],g[3])
        vals = 'stat={}, p={}'.format(stat,p)
        if p < .05:
            moodslist[c] = p
        else:
            pass

__SVD Truncate: To find most important features__

In [41]:
svd = TruncatedSVD(n_components=20, n_iter=7, random_state=42)
svd.fit(X)

TruncatedSVD(algorithm='randomized', n_components=20, n_iter=7,
       random_state=42, tol=0.0)

In [42]:
best_features = list(moodslist.keys())

__Reduce Features (Unused): 14,665 to n_components __

__Split Data to Train/Test__

_Using TrainTestSplit (Unused)_

_Even Distribution Sampling_

In [43]:
# evenly_distributed_test = [60 japanese,60 english, 60 chinese, 60 korean]
rs=33
japset = features[features['language'] == 'Japanese'].sample(n=100, random_state=rs)
korset = features[features['language'] == 'Korean'].sample(n=100, random_state=rs)
chiset = features[features['language'] == 'Traditional Chinese'].sample(n=100, random_state=rs)
engset = features[features['language'] == 'English'].sample(n=10, random_state=rs)
testset = [japset,korset,chiset,engset]

In [44]:
test_data = pd.concat(testset)
y_test = test_data['language'].values.reshape(-1, 1).ravel()
X_test = test_data[best_features]

In [ ]:
train_data = features[~features.isin(test_data)].dropna()
y_train = train_data['language'].values.reshape(-1, 1).ravel()
X_train = train_data[best_features]
# X_train = svd.transform(train_data[train_data.columns[~train_data.columns.str.contains('language')]])

<a id="seca"></a>

__Create Function for Comparing Models__

In [ ]:
cols = ['name','time','total','precision','recall','f1']

model_set = pd.DataFrame(columns=cols)
models_stored = []
pattern = "%.2f"

lang = list(set(y_train))

In [ ]:
def run_model(model,name):
    global model_set
    m = model
    m.fit(X_train, y_train)
    start = time.time()

    total_score = m.score(X_test,y_test)
    pscore = [pattern % i for i in list(metrics.precision_score(y_test, m.predict(X_test),labels=lang,average=None))]
    rscore = [pattern % i for i in list(metrics.recall_score(y_test, m.predict(X_test),labels=lang,average=None))]
    fscore = [pattern % i for i in list(metrics.f1_score(y_test, m.predict(X_test),labels=lang,average=None))]
    end = time.time()
    t= pattern % (end - start)

    r = dict(zip(cols,[name,t,total_score,pscore,rscore,fscore]))
    print('Check for Overfitting: {}\n'.format(m.score(X_train,y_train)))
    print('Test Score is: {}\n'.format(total_score))
    print(classification_report(y_test, m.predict(X_test)))
    
    model_set = model_set.append(r,ignore_index=True)
    return r,m

<a id="seca"></a>

### <span style="color:darkred">A. LR - Logistic Regression</span>  <a href='#top'>(top)</a>

> Target is binary so logistic regression will operate on probabilities

In [ ]:
%%time
lreg_data,lreg = run_model(linear_model.LogisticRegression(),'Logistic Regression')

<a id="secb1"></a>

<a id="sece"></a>

### <span style="color:darkred">E. K Nearest Neighbors</span>  <a href='#top'>(top)</a>

> Can handle discrete values for target <br>Quantitative values are limited (not continuous) and might be problematic for nearest neighbors

In [ ]:
%%time
neighbors_data,neighbors = run_model(KNeighborsClassifier(n_neighbors=10),'K Nearest Neighbor')

<a id="secf"></a>

### <span style="color:darkred">F. Naive Bayes - Bernoulli</span>  <a href='#top'>(top)</a>

In [ ]:
%%time
bnb_data,bnb = run_model(GaussianNB(),'Naive Bayes - Bernoulli')

<a id="secg"></a>

### <span style="color:darkred">G. Decision Tree</span>  <a href='#top'>(top)</a>

In [29]:
%%time
dt_data,dt = run_model(tree.DecisionTreeClassifier(criterion='entropy',max_depth=6),'Decision Tree')

Check for Overfitting: 0.744862518089725

Test Score is: 0.4838709677419355

                     precision    recall  f1-score   support

            English       1.00      0.10      0.18        10
           Japanese       0.40      0.94      0.56       100
             Korean       0.87      0.20      0.33       100
Traditional Chinese       0.66      0.35      0.46       100

        avg / total       0.66      0.48      0.44       310

CPU times: user 1.72 s, sys: 171 ms, total: 1.89 s
Wall time: 1.92 s


In [30]:
# Render tree.
dot_data = tree.export_graphviz(
    dt, 
    out_file=None,
    feature_names=X_train.columns,
    label= 'root',
    proportion=False,
    rounded=True,
    class_names=lang,
    filled=True
)

graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

graph.write_png('decision_tree.png')

True

_Good visualization of important features and presentation of entropy weighting_

<a id="sech"></a>

### <span style="color:darkred">H. Random Forest</span>  <a href='#top'>(top)</a>

> Runs decision tree multiple times for best output <br>Longest processing time

In [31]:
%%time
rf_data,rf = run_model(ensemble.RandomForestClassifier(n_estimators=80,
                                                       criterion='entropy',
                                                       max_features=len(X_train.columns),
                                                       max_depth=6),'Random Forest')

Check for Overfitting: 0.7549204052098408

Test Score is: 0.4774193548387097

                     precision    recall  f1-score   support

            English       0.00      0.00      0.00        10
           Japanese       0.39      0.98      0.56       100
             Korean       0.88      0.15      0.26       100
Traditional Chinese       0.80      0.35      0.49       100

        avg / total       0.67      0.48      0.42       310

CPU times: user 1min 25s, sys: 702 ms, total: 1min 26s
Wall time: 1min 27s


In [ ]:
rf.feature_importances_
importance = dict(list(zip(X.columns,rf.feature_importances_)))
importance_sorted = sorted(importance, key=importance.get, reverse=True)
# for r in importance_sorted:
#     if importance[r] >0:
#         print(r, importance[r])
print(importance_sorted[0:100])

<a id="sec2"></a>

# <span style="color:darkblue">2. Model Comparison</span>  <a href='#top'>(top)</a>

In [32]:
model_set.columns = ['name','time','total','prec: | JA | CH | KO | EN |','rec: | JA | CH | KO | EN |','f1: | JA | CH | KO | EN |']
model_set

,name,time,total,prec: | JA | CH | KO | EN |,rec: | JA | CH | KO | EN |,f1: | JA | CH | KO | EN |
0,Logistic Regression,0.04,0.441935,"[1.00, 0.38, 0.68, 1.00]","[0.01, 0.96, 0.39, 0.10]","[0.02, 0.55, 0.50, 0.18]"
1,K Nearest Neighbor,1.17,0.316129,"[0.00, 0.32, 0.20, 0.00]","[0.00, 0.96, 0.02, 0.00]","[0.00, 0.48, 0.04, 0.00]"
2,Naive Bayes - Bernoulli,0.12,0.329032,"[0.34, 0.35, 0.25, 0.00]","[0.29, 0.67, 0.06, 0.00]","[0.31, 0.46, 0.10, 0.00]"
3,Naive Bayes - Bernoulli,0.13,0.329032,"[0.34, 0.35, 0.25, 0.00]","[0.29, 0.67, 0.06, 0.00]","[0.31, 0.46, 0.10, 0.00]"
4,Decision Tree,0.02,0.483871,"[0.87, 0.40, 0.66, 1.00]","[0.20, 0.94, 0.35, 0.10]","[0.33, 0.56, 0.46, 0.18]"
5,Random Forest,0.06,0.477419,"[0.88, 0.39, 0.80, 0.00]","[0.15, 0.98, 0.35, 0.00]","[0.26, 0.56, 0.49, 0.00]"


-----

Regularization